In [ ]:
# %reload_ext jupyter_ai
# %load_ext jupyter_ai_magics

In [ ]:
# %ai chatgpt --format code
# The best regression for a pandas dataframe

In [ ]:
#!pip install pandas numpy matplotlib xgboost scikit-learn tqdm

In [3]:
!pip install scikit-learn-intelex

## Optuna

* https://optuna.org/
* https://practicaldatascience.co.uk/machine-learning/how-to-tune-an-xgbregressor-model-with-optuna
* https://practicaldatascience.co.uk/machine-learning/how-to-use-optuna-for-xgboost-hyperparameter-tuning
* https://medium.com/optuna/using-optuna-to-optimize-xgboost-hyperparameters-63bfcdfd3407
* https://randomrealizations.com/posts/xgboost-parameter-tuning-with-optuna/

In [4]:
! pip install optuna optuna-dashboard --quiet

In [2]:
OPTUNA_DB = "sqlite:///optuna.sqlite3"
get_ipython().system_raw(f"optuna-dashboard {OPTUNA_DB} &")

Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/sqlalchemy/engine/base.py", line 1967, in _exec_single_context
    self.dialect.do_execute(
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/sqlalchemy/engine/default.py", line 924, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: version_info

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/optuna/storages/_rdb/storage.py", line 72, in _create_scoped_session
    yield session
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/optuna/storages/_rdb/storage.py", line 1042, in _init_version_info_model


# Autogluon
* https://auto.gluon.ai/stable/tutorials/tabular/tabular-feature-engineering.html
* https://auto.gluon.ai/stable/tutorials/tabular/advanced/index.html

In [3]:
!pip install -U autogluon ipywidgets
# !pip uninstall lightgbm -y
# !pip install lightgbm --install-option=--gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.8 MB/s eta 0:00:0031m2.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.5 MB/s eta 0:00:006.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 4.0.10
    Uninstalling widgetsnbextension-4.0.10:
      Successfully uninstalled widgetsnbextension-4.0.10
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.10
    Uninstalling jupyterlab_widgets-3.0.10:
      Successfully uninstalled jupyterlab_widgets-3.0.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 8.1.2
    Uninstalling ipywidgets-8.1.2:
      Successfully uninstalled ipywidgets-8.1.2


In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

# MLFlow
* https://www.kaggle.com/code/sharanharsoor/mlflow-end-to-end-ml-models

In [5]:
!pip install mlflow 'mxnet<=1.9.1' --quiet
!pip install pyngrok --quiet
get_ipython().system_raw("mlflow ui --port 5555 &")

In [7]:
import mlflow

mlflow.set_tracking_uri(uri="http://127.0.0.1:5555")
mlflow.autolog()
mlflow.lightgbm.autolog()
mlflow.xgboost.autolog()
# mlflow.gluon.autolog()

# !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd


2024/06/05 12:11:51 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


# Weight and Biases
* https://colab.research.google.com/github/wandb/examples/blob/master/colabs/boosting/Credit_Scorecards_with_XGBoost_and_W%26B.ipynb#scrollTo=agAgfdIeuPdXj


In [8]:
!pip install --upgrade -q wandb

In [9]:
import wandb
WANDB_PROJECT = "intive-interstellar-sales-showdown"

In [10]:
try:
    from kaggle_secrets import UserSecretsClient

    client = UserSecretsClient()
    wandb.login(wandb_api=client.get_secret("wandb_api"))
except ModuleNotFoundError:
    print("Falling back to environmenet variable")
    wandb.login()

Falling back to environmenet variable


wandb: Currently logged in as: gonzalo-matheu-intive. Use `wandb login --relogin` to force relogin


# Links
 * https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html
 * https://scikit-learn.org/stable/modules/cross_validation.html#computing-cross-validated-metrics

In [11]:
import datetime
import pickle
import warnings
from dataclasses import dataclass
from datetime import date
from getpass import getpass
from pathlib import Path

In [12]:
import lightgbm
import lightgbm as lgbm
import mlflow
import mlflow.lightgbm
import mlflow.xgboost
import numpy as np
import optuna
import optuna.visualization as ov
import pandas as pd
import xgboost
import xgboost as xgb
from numpy import mean, std
from optuna.visualization import plot_intermediate_values
from pandas import read_csv
from pyngrok import ngrok
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, mean_absolute_error, mean_squared_error
from sklearn.model_selection import (
    RepeatedKFold,
    RepeatedStratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
)
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [13]:
from pathlib import Path

dataset_name = "original"
output_path = "."

test_path = f"{dataset_name}_test.csv"
train_path = f"{dataset_name}_train.csv"
if not Path(test_path).exists():
  test_path = f'/kaggle/input/{dataset_name}-dataset/{dataset_name}_test.csv'
  train_path = f'/kaggle/input/{dataset_name}-dataset/{dataset_name}_train.csv'
test_csv = pd.read_csv(test_path)
train_csv = pd.read_csv(train_path)
# sample_submission = pd.read_csv('sample_submission.csv')
train_csv


,id,price,sale_date,cabins,decks,bathrooms,producer,created_date,refitted_date,total_area,...,weight_distribution_x,weight_distribution_y,engine_thrust,radar,bow_thruster,autopilot,solar_panels,navi_quality,state,overall_quality
0,1,1845000.0,2122-11-28,3,1.0,1,59,2106/08,NaN,364.6,...,-0.070052,0.016326,1899.6,0,1,0,0,0,4,8
1,2,861600.0,2122-11-28,3,1.0,3,24,2115/08,NaN,998.3,...,0.104948,0.105376,1550.3,0,1,1,0,0,4,7
2,3,1446000.0,2122-11-28,4,2.0,2,24,2114/12,NaN,1793.4,...,0.110948,0.092526,2692.4,0,0,0,0,0,3,9
3,4,2250000.0,2122-11-28,3,1.0,2,51,2108/01,NaN,327.5,...,-0.082052,-0.005674,1496.6,0,1,1,0,2,4,8
4,5,1200000.0,2122-11-28,4,1.5,2,41,2112/11,NaN,2144.2,...,0.026948,0.149176,2507.2,0,1,1,0,0,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,15125,1005000.0,2123-12-10,3,1.0,2,14,2114/01,NaN,2243.3,...,-0.063052,0.116376,1425.4,0,0,1,0,0,4,7
15125,15126,1080000.0,2123-12-10,4,2.0,2,20,2115/11,NaN,364.5,...,0.036448,0.079526,1150.8,0,1,1,0,0,3,7
15126,15127,2565000.0,2123-12-10,3,1.0,1,43,2110/01,NaN,200.0,...,-0.043552,-0.062674,1919.0,0,0,0,0,0,3,7
15127,15128,1336500.0,2123-12-20,2,1.0,1,50,2105/05,NaN,333.5,...,-0.032052,-0.065674,771.2,0,0,0,0,0,3,6


In [14]:
results = pd.DataFrame(columns=["dataset", "method", "variant", "rmse", "mse", "mae", "timestamp"])
results

,dataset,method,variant,rmse,mse,mae,timestamp


In [15]:
train_csv

,id,price,sale_date,cabins,decks,bathrooms,producer,created_date,refitted_date,total_area,...,weight_distribution_x,weight_distribution_y,engine_thrust,radar,bow_thruster,autopilot,solar_panels,navi_quality,state,overall_quality
0,1,1845000.0,2122-11-28,3,1.0,1,59,2106/08,NaN,364.6,...,-0.070052,0.016326,1899.6,0,1,0,0,0,4,8
1,2,861600.0,2122-11-28,3,1.0,3,24,2115/08,NaN,998.3,...,0.104948,0.105376,1550.3,0,1,1,0,0,4,7
2,3,1446000.0,2122-11-28,4,2.0,2,24,2114/12,NaN,1793.4,...,0.110948,0.092526,2692.4,0,0,0,0,0,3,9
3,4,2250000.0,2122-11-28,3,1.0,2,51,2108/01,NaN,327.5,...,-0.082052,-0.005674,1496.6,0,1,1,0,2,4,8
4,5,1200000.0,2122-11-28,4,1.5,2,41,2112/11,NaN,2144.2,...,0.026948,0.149176,2507.2,0,1,1,0,0,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,15125,1005000.0,2123-12-10,3,1.0,2,14,2114/01,NaN,2243.3,...,-0.063052,0.116376,1425.4,0,0,1,0,0,4,7
15125,15126,1080000.0,2123-12-10,4,2.0,2,20,2115/11,NaN,364.5,...,0.036448,0.079526,1150.8,0,1,1,0,0,3,7
15126,15127,2565000.0,2123-12-10,3,1.0,1,43,2110/01,NaN,200.0,...,-0.043552,-0.062674,1919.0,0,0,0,0,0,3,7
15127,15128,1336500.0,2123-12-20,2,1.0,1,50,2105/05,NaN,333.5,...,-0.032052,-0.065674,771.2,0,0,0,0,0,3,6


In [16]:
train_csv.columns

Index(['id', 'price', 'sale_date', 'cabins', 'decks', 'bathrooms', 'producer',
       'created_date', 'refitted_date', 'total_area', 'crew_area',
       'cabins_area', 'maintenance_area', 'crew_area_coef', 'total_area_coef',
       'weight_distribution_x', 'weight_distribution_y', 'engine_thrust',
       'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality',
       'state', 'overall_quality'],
      dtype='object')

In [17]:
test_csv

,id,sale_date,cabins,decks,bathrooms,producer,created_date,refitted_date,total_area,crew_area,...,weight_distribution_x,weight_distribution_y,engine_thrust,radar,bow_thruster,autopilot,solar_panels,navi_quality,state,overall_quality
0,15130,2122-11-28,4,2.0,3,41,2116/06,NaN,361.6,155.5,...,0.026448,0.116076,1324.3,0,1,1,0,0,3,8
1,15131,2122-11-28,3,1.0,2,3,2114/07,NaN,541.7,104.5,...,-0.029552,0.103176,2408.2,0,0,0,0,0,4,8
2,15132,2122-11-28,4,2.0,2,37,2117/11,NaN,167.2,143.0,...,0.042948,-0.106724,1111.5,0,1,1,0,0,3,8
3,15133,2122-11-28,3,1.0,2,52,2107/03,NaN,237.5,130.0,...,-0.078052,-0.061624,1291.7,0,0,0,0,0,4,9
4,15134,2122-11-28,4,2.0,2,29,2117/03,NaN,2149.9,165.5,...,0.056448,-0.081574,2965.0,0,0,1,0,0,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6479,21609,2123-12-09,4,2.0,3,36,2117/09,NaN,902.1,124.5,...,-0.098552,0.071976,1528.8,0,1,1,0,0,3,8
6480,21610,2123-12-10,4,1.0,1,51,2109/02,NaN,292.5,99.0,...,-0.073552,-0.000324,1597.0,0,1,0,0,1,4,8
6481,21611,2123-12-10,5,2.0,3,43,2117/11,NaN,267.7,166.0,...,-0.043552,-0.047074,2173.0,0,1,1,0,0,3,9
6482,21612,2123-12-10,4,2.0,2,24,2117/09,NaN,589.0,129.0,...,0.101948,0.097126,2963.6,0,0,0,0,0,3,9


In [18]:
all_features = list(test_csv.columns)
test_csv.columns

Index(['id', 'sale_date', 'cabins', 'decks', 'bathrooms', 'producer',
       'created_date', 'refitted_date', 'total_area', 'crew_area',
       'cabins_area', 'maintenance_area', 'crew_area_coef', 'total_area_coef',
       'weight_distribution_x', 'weight_distribution_y', 'engine_thrust',
       'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality',
       'state', 'overall_quality'],
      dtype='object')

In [74]:
patch_sklearn()

warnings.filterwarnings(
    action="ignore",
    message=".*Could not infer format.*",
)

warnings.filterwarnings(
    action="ignore",
    message=".*autologging encountered a warning.*",
)

warnings.filterwarnings(
    action="ignore",
    message=".*No visible GPU is found.*",
)
warnings.filterwarnings(
    action="ignore",
    message=".*deprecated binary model.*",
)


def load_numeric(file, datetime_to_numeric=True):
    csv = pd.read_csv(file)

    csv["weight_distribution_y"] = pd.to_numeric(csv["weight_distribution_y"], errors="coerce")
    csv = csv.dropna(subset=['weight_distribution_y'])

    csv["created_date"] = pd.to_datetime(csv["created_date"])
    csv["refitted_date"] = pd.to_datetime(csv["refitted_date"])
    csv["refitted_date"] = csv["refitted_date"].fillna(csv["created_date"])

    csv["sale_date"] = pd.to_datetime(csv["sale_date"])
    age_baseline = max(
        csv["created_date"].max(), csv["refitted_date"].max(), csv["sale_date"].max()
    )

    calculate_age = (
        lambda x: age_baseline.year
        - x.year
        - ((age_baseline.month, age_baseline.day) < (x.month, x.day))
    )
    csv["created_age"] = csv["created_date"].apply(calculate_age)
    csv["refitted_age"] = csv["refitted_date"].apply(calculate_age)
    csv["sale_age"] = csv["sale_date"].apply(calculate_age)

    if datetime_to_numeric:
        csv["created_date"] = pd.to_numeric(csv["created_date"])
        csv["refitted_date"] = pd.to_numeric(csv["refitted_date"])
        csv["sale_date"] = pd.to_numeric(csv["sale_date"])
    print(csv.select_dtypes(include=["object"]).dtypes)
    return csv


def load_train(datetime_to_numeric=True):
    return load_numeric(train_path, datetime_to_numeric=datetime_to_numeric)


def load_test(datetime_to_numeric=True):
    return load_numeric(test_path, datetime_to_numeric=datetime_to_numeric)


def store_submission(alias, test_data, model=None):
    now = datetime.datetime.now().replace(microsecond=0).isoformat().replace(":", "")
    submissions = Path(output_path) / "submissions"
    submissions.mkdir(exist_ok=True)
    filename = f"{submissions}/{now}_{alias}_{dataset_name}_submission"
    if "id" not in test_data.columns:
        test_data["id"] = pd.read_csv("test.csv")["id"]
    test_data[["id", "price"]].to_csv(f"{filename}.csv", index=False)

    if model is not None:
        model_path = f"{filename}.pkl"
        pickle.dump(model, open(model_path, "wb"))


def predict_and_store(alias, model, test_x):
    test_data = load_test()
    test_data["price"] = model.predict(test_x)
    store_submission(alias, test_data, model)
    return test_data


@dataclass
class LocalDataset:
    X: pd.DataFrame
    y: pd.DataFrame
    X_train: pd.DataFrame
    X_test: pd.DataFrame
    y_train: pd.DataFrame
    y_test: pd.DataFrame
    test_data: pd.DataFrame
    train_data: pd.DataFrame
    features: list[str]
    feature_generator: AutoMLPipelineFeatureGenerator


@dataclass
class ModelTrain:
    X = None
    y = None
    X_train = None
    X_test = None
    y_train = None
    y_test = None
    features = None
    model = None


def load_split(
    datetime_to_numeric=True,
    test_size=0.2,
    features=None,
    drop=None,
    standard_scaler=True,
    feature_generator: AutoMLPipelineFeatureGenerator =None
):
    train_data = load_train(datetime_to_numeric=datetime_to_numeric)
    test_data = load_test(datetime_to_numeric=datetime_to_numeric)
    
    df = train_data
    X = df.drop(["price"], axis=1)
    if drop is not None:
        X = df.drop(drop, axis=1)
        test_data = test_data.drop(drop, axis=1)
    if features is None:
        features = list(X.columns)
    else:
        X = X[features]
        test_data = test_data[features]
        print(f"Keeping features {features}")
    y = df["price"]
    
    if feature_generator is not None:
        print(feature_generator)
        X = feature_generator.fit_transform(X, y)
        test_data = feature_generator.transform(test_data)
        train_data = feature_generator.transform(X)
        train_data["price"] = y
        # test_data = feature_generator.transform(test_data)
        print(feature_generator.print_generator_info())


    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=1
    )

    X_train = pd.DataFrame(X_train, columns=X.columns)
    X_test = pd.DataFrame(X_test, columns=X.columns)
    test_data = pd.DataFrame(test_data, columns=X.columns)
    train_data = pd.DataFrame(train_data[X.columns], columns=X.columns)
    if standard_scaler:
        scaler = StandardScaler()
        X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
        test_data = pd.DataFrame(scaler.transform(test_data), columns=X.columns)
        train_data = pd.DataFrame(
            scaler.transform(train_data[X.columns]), columns=X.columns
        )
        
    train_data["price"] = y

    return LocalDataset(
        X, y, X_train, X_test, y_train, y_test, test_data, train_data, features, feature_generator
    )


def calculate_metrics(ld: LocalDataset, model, method, variant="", y_pred=None):
    if y_pred is None:
        y_pred = model.predict(ld.X_test)
    now = datetime.datetime.now().replace(microsecond=0).isoformat().replace(":", "")
    print("MSE: ", mean_squared_error(ld.y_test, y_pred))
    print("RMSE: ", np.sqrt(mean_squared_error(ld.y_test, y_pred)))
    print("MAE: ", mean_absolute_error(ld.y_test, y_pred))
    metrics = {
        "rmse": np.sqrt(mean_squared_error(ld.y_test, y_pred)),
        "mse": mean_squared_error(ld.y_test, y_pred),
        "mae": mean_absolute_error(ld.y_test, y_pred),
    }
    results.loc[len(results) + 1] = {
        "dataset": dataset_name,
        "method": method,
        "variant": variant,
        "rmse": np.sqrt(mean_squared_error(ld.y_test, y_pred)),
        "mse": mean_squared_error(ld.y_test, y_pred),
        "mae": mean_absolute_error(ld.y_test, y_pred),
        "timestamp": now
    }
    mlflow.log_metrics(metrics)
    results.to_csv(f"results.csv", index=False)
    return results.sort_values(by="rmse")


ld = load_split()

ld.train_data.info()

Series([], dtype: object)
Series([], dtype: object)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15129 entries, 0 to 15128
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     15129 non-null  float64
 1   sale_date              15129 non-null  float64
 2   cabins                 15129 non-null  float64
 3   decks                  15129 non-null  float64
 4   bathrooms              15129 non-null  float64
 5   producer               15129 non-null  float64
 6   created_date           15129 non-null  float64
 7   refitted_date          15129 non-null  float64
 8   total_area             15129 non-null  float64
 9   crew_area              15129 non-null  float64
 10  cabins_area            15129 non-null  float64
 11  maintenance_area       15129 non-null  float64
 12  crew_area_coef         15129 non-null  float64
 13  total_area_coef        15129 non-null  float64
 14  we

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [75]:
ld.X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15129 entries, 0 to 15128
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     15129 non-null  int64  
 1   sale_date              15129 non-null  int64  
 2   cabins                 15129 non-null  int64  
 3   decks                  15129 non-null  float64
 4   bathrooms              15129 non-null  int64  
 5   producer               15129 non-null  int64  
 6   created_date           15129 non-null  int64  
 7   refitted_date          15129 non-null  int64  
 8   total_area             15129 non-null  float64
 9   crew_area              15129 non-null  float64
 10  cabins_area            15129 non-null  float64
 11  maintenance_area       15129 non-null  float64
 12  crew_area_coef         15129 non-null  float64
 13  total_area_coef        15129 non-null  float64
 14  weight_distribution_x  15129 non-null  float64
 15  we

In [76]:
feature_generator = AutoMLPipelineFeatureGenerator()
auto_ld = load_split(datetime_to_numeric=False, standard_scaler=False, feature_generator=feature_generator)
auto_ld.X.info()

Series([], dtype: object)
Series([], dtype: object)
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15129 entries, 0 to 15128
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       15129 non-null  int64  
 1   cabins                   15129 non-null  int64  
 2   decks                    15129 non-null  float64
 3   bathrooms                15129 non-null  int64  
 4   producer                 15129 non-null  int64  
 5   total_area               15129 non-null  float64
 6   crew_area                15129 non-null  float64
 7   cabins_area              15129 non-null  float64
 8   maintenance_area         15129 non-null  float64
 9   crew_area_coef           15129 non-null  float64
 10  total_area_coef          15129 non-null  float64
 11  weight_distribution_x    15129 non-null  float64
 12  weight_distribution_y    15129 non-null  float64
 13  engine_thrust      

# Sample code

In [86]:
def execute_sample_code(features=["cabins", "decks", "bathrooms", "radar"], ld: LocalDataset = None, variant=""):
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error

    with mlflow.start_run(
        run_name=f"linear-{variant}",
        tags={"version": "v1", "library": "sklearn", "optimization": "n/a"},
        description="sklearn",
    ):
        # load training and test data
        if ld is None:
            ld = load_split(features=features)
        train_data = ld.X_train
        # choose 4 features and price as training target
        # features = ['cabins', 'decks', 'bathrooms', 'radar']
        # train_x = train_data[features]
        # train_y = ld.y_train

        # simple linear regression model
        # price = a * cabins + b * decs + c * bathrooms + d * radar + e
        model = LinearRegression().fit(ld.X_train, ld.y_train)

        # predict price on test data
        # test_x = ld.test_data[features]
        # test_data = predict_and_store('linear_regression', model, test_x)

        # y_pred = model.predict(ld.X_test)
        # print('MSE: ', mean_squared_error(ld.y_test, y_pred))
        # print('RMSE: ', np.sqrt(mean_squared_error(ld.y_test, y_pred)))
        # print('MAE: ', np.sqrt(mean_absolute_error(ld.y_test, y_pred)))
        calculate_metrics(ld, model, method="linear", variant=variant)

        # mlflow.sklearn.log_model(sk_model=model, input_example=ld.X_train.sample(10))
        # mlflow.log_params(params)

        return model, ld.X_train


sample_model, sample_x = None, None
sample_model, sample_x = execute_sample_code(variant="demo")

Series([], dtype: object)
Series([], dtype: object)
Keeping features ['cabins', 'decks', 'bathrooms', 'radar']
MSE:  780428937023.6322
RMSE:  883418.8910271459
MAE:  610430.3268213463


# SelectKBest features

* https://lifewithdata.com/2022/03/19/feature-selection-with-selectkbest-in-scikit-learn/

In [87]:
def select_features(k=10, ld: LocalDataset = None):
    from sklearn.feature_selection import SelectKBest, chi2
    from sklearn.preprocessing import MinMaxScaler

    # Assuming X and y are your input features and target labels, respectively
    # Select the top 10 features
    selector = SelectKBest(score_func=chi2, k=k)

    df = load_train()
    if ld is not None:
        df = ld.train_data
    X = df[[col for col in df.columns if col != "price"]]
    y = df["price"]

    print(X.dtypes)

    sc = MinMaxScaler()
    X_sc = sc.fit_transform(X)

    # X_new contains the selected features
    X_new = selector.fit_transform(X_sc, y)

    kept_features = pd.DataFrame({"columns": X.columns, "Kept": selector.get_support()})
    #     print(kept_features)

    new_train_csv = X.iloc[:, selector.get_support()]
    print(list(new_train_csv.columns))
    return list(new_train_csv.columns), new_train_csv


kept_features, train_csv_kept_features = select_features()
train_csv_kept_features

Series([], dtype: object)
id                         int64
sale_date                  int64
cabins                     int64
decks                    float64
bathrooms                  int64
producer                   int64
created_date               int64
refitted_date              int64
total_area               float64
crew_area                float64
cabins_area              float64
maintenance_area         float64
crew_area_coef           float64
total_area_coef          float64
weight_distribution_x    float64
weight_distribution_y    float64
engine_thrust            float64
radar                      int64
bow_thruster               int64
autopilot                  int64
solar_panels               int64
navi_quality               int64
state                      int64
overall_quality            int64
created_age                int64
refitted_age               int64
sale_age                   int64
dtype: object
['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_

,decks,maintenance_area,weight_distribution_y,radar,bow_thruster,autopilot,solar_panels,navi_quality,refitted_age,sale_age
0,1.0,50.0,0.016326,0,1,0,0,0,17,1
1,1.0,38.0,0.105376,0,1,1,0,0,8,1
2,2.0,0.0,0.092526,0,0,0,0,0,9,1
3,1.0,47.5,-0.005674,0,1,1,0,2,15,1
4,1.5,66.5,0.149176,0,1,1,0,0,11,1
...,...,...,...,...,...,...,...,...,...,...
15124,1.0,0.0,0.116376,0,0,1,0,0,9,0
15125,2.0,0.0,0.079526,0,1,1,0,0,8,0
15126,1.0,30.0,-0.062674,0,0,0,0,0,13,0
15127,1.0,33.5,-0.065674,0,0,0,0,0,18,0


In [99]:
feature_generator = AutoMLPipelineFeatureGenerator()
select_features(ld=load_split(datetime_to_numeric=False, standard_scaler=False, feature_generator=feature_generator))[1]

Series([], dtype: object)
Series([], dtype: object)
None
Series([], dtype: object)
id                           int64
cabins                       int64
decks                      float64
bathrooms                    int64
producer                     int64
total_area                 float64
crew_area                  float64
cabins_area                float64
maintenance_area           float64
crew_area_coef             float64
total_area_coef            float64
weight_distribution_x      float64
weight_distribution_y      float64
engine_thrust              float64
radar                         int8
bow_thruster                  int8
autopilot                     int8
solar_panels                  int8
navi_quality                 int64
state                        int64
overall_quality              int64
created_age                  int64
refitted_age                 int64
sale_age                      int8
sale_date                    int64
sale_date.year               int64
sale_da

,decks,maintenance_area,weight_distribution_y,radar,bow_thruster,autopilot,solar_panels,navi_quality,sale_age,sale_date.month
0,1.0,50.0,0.016326,0,1,0,0,0,1,11
1,1.0,38.0,0.105376,0,1,1,0,0,1,11
2,2.0,0.0,0.092526,0,0,0,0,0,1,11
3,1.0,47.5,-0.005674,0,1,1,0,2,1,11
4,1.5,66.5,0.149176,0,1,1,0,0,1,11
...,...,...,...,...,...,...,...,...,...,...
15124,1.0,0.0,0.116376,0,0,1,0,0,0,12
15125,2.0,0.0,0.079526,0,1,1,0,0,0,12
15126,1.0,30.0,-0.062674,0,0,0,0,0,0,12
15127,1.0,33.5,-0.065674,0,0,0,0,0,0,12


In [89]:
_ = execute_sample_code(features=kept_features, variant="kbest10")

Series([], dtype: object)
Series([], dtype: object)
Keeping features ['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']
MSE:  682649835361.2692
RMSE:  826226.2616022739
MAE:  524141.9240594843


In [90]:
train_csv['weight_distribution_y'].describe()
# pd.to_numeric(train_csv["weight_distribution_y"], errors="raise")

train_csv.iloc[7578]

train_csv.isnull().any(axis=0)
df = train_csv
df["weight_distribution_y"] = pd.to_numeric(df["weight_distribution_y"], errors="coerce")
df = train_csv.dropna(subset=['weight_distribution_y'])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15129 entries, 0 to 15128
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     15129 non-null  int64  
 1   price                  15129 non-null  float64
 2   sale_date              15129 non-null  object 
 3   cabins                 15129 non-null  int64  
 4   decks                  15129 non-null  float64
 5   bathrooms              15129 non-null  int64  
 6   producer               15129 non-null  int64  
 7   created_date           15129 non-null  object 
 8   refitted_date          656 non-null    object 
 9   total_area             15129 non-null  float64
 10  crew_area              15129 non-null  float64
 11  cabins_area            15129 non-null  float64
 12  maintenance_area       15129 non-null  float64
 13  crew_area_coef         15129 non-null  float64
 14  total_area_coef        15129 non-null  float64
 15  we

In [91]:
_ = execute_sample_code(features=select_features(k=5)[0], variant="kbest5")

Series([], dtype: object)
id                         int64
sale_date                  int64
cabins                     int64
decks                    float64
bathrooms                  int64
producer                   int64
created_date               int64
refitted_date              int64
total_area               float64
crew_area                float64
cabins_area              float64
maintenance_area         float64
crew_area_coef           float64
total_area_coef          float64
weight_distribution_x    float64
weight_distribution_y    float64
engine_thrust            float64
radar                      int64
bow_thruster               int64
autopilot                  int64
solar_panels               int64
navi_quality               int64
state                      int64
overall_quality            int64
created_age                int64
refitted_age               int64
sale_age                   int64
dtype: object
['radar', 'bow_thruster', 'solar_panels', 'navi_quality', 'sale_age']

In [92]:
_ = execute_sample_code(features=select_features(k=15)[0], variant="kbest15")

Series([], dtype: object)
id                         int64
sale_date                  int64
cabins                     int64
decks                    float64
bathrooms                  int64
producer                   int64
created_date               int64
refitted_date              int64
total_area               float64
crew_area                float64
cabins_area              float64
maintenance_area         float64
crew_area_coef           float64
total_area_coef          float64
weight_distribution_x    float64
weight_distribution_y    float64
engine_thrust            float64
radar                      int64
bow_thruster               int64
autopilot                  int64
solar_panels               int64
navi_quality               int64
state                      int64
overall_quality            int64
created_age                int64
refitted_age               int64
sale_age                   int64
dtype: object
['id', 'sale_date', 'decks', 'producer', 'cabins_area', 'maintenance_

In [93]:
_ = execute_sample_code(features=select_features(k=20)[0], variant="kbest20")

Series([], dtype: object)
id                         int64
sale_date                  int64
cabins                     int64
decks                    float64
bathrooms                  int64
producer                   int64
created_date               int64
refitted_date              int64
total_area               float64
crew_area                float64
cabins_area              float64
maintenance_area         float64
crew_area_coef           float64
total_area_coef          float64
weight_distribution_x    float64
weight_distribution_y    float64
engine_thrust            float64
radar                      int64
bow_thruster               int64
autopilot                  int64
solar_panels               int64
navi_quality               int64
state                      int64
overall_quality            int64
created_age                int64
refitted_age               int64
sale_age                   int64
dtype: object
['id', 'sale_date', 'decks', 'producer', 'created_date', 'refitted_da

In [94]:
feature_generator = AutoMLPipelineFeatureGenerator()
ld = load_split(datetime_to_numeric=False, standard_scaler=False, feature_generator=feature_generator)
_ = execute_sample_code(ld=ld, features=select_features(k=20, ld=ld)[0], variant="kbest20_autofeatures")

Series([], dtype: object)
Series([], dtype: object)
None
Series([], dtype: object)
id                           int64
cabins                       int64
decks                      float64
bathrooms                    int64
producer                     int64
total_area                 float64
crew_area                  float64
cabins_area                float64
maintenance_area           float64
crew_area_coef             float64
total_area_coef            float64
weight_distribution_x      float64
weight_distribution_y      float64
engine_thrust              float64
radar                         int8
bow_thruster                  int8
autopilot                     int8
solar_panels                  int8
navi_quality                 int64
state                        int64
overall_quality              int64
created_age                  int64
refitted_age                 int64
sale_age                      int8
sale_date                    int64
sale_date.year               int64
sale_da

In [105]:
feature_generator = AutoMLPipelineFeatureGenerator()
ld = load_split(datetime_to_numeric=False, standard_scaler=False, feature_generator=feature_generator)
_ = execute_sample_code(ld=ld, features=select_features(k=28, ld=ld)[0], variant="kbest28_autofeatures")

Series([], dtype: object)
Series([], dtype: object)
None
Series([], dtype: object)
id                           int64
cabins                       int64
decks                      float64
bathrooms                    int64
producer                     int64
total_area                 float64
crew_area                  float64
cabins_area                float64
maintenance_area           float64
crew_area_coef             float64
total_area_coef            float64
weight_distribution_x      float64
weight_distribution_y      float64
engine_thrust              float64
radar                         int8
bow_thruster                  int8
autopilot                     int8
solar_panels                  int8
navi_quality                 int64
state                        int64
overall_quality              int64
created_age                  int64
refitted_age                 int64
sale_age                      int8
sale_date                    int64
sale_date.year               int64
sale_da

# SkLearn Pipeline

https://machinelearningmastery.com/feature-extraction-on-tabular-data/

In [106]:

train_data = load_train()
test_data = load_test()

df = train_data
X = df[[col for col in df.columns if col != "price"]]
# features = list(new_train_csv.columns)
# X = df[features]
y = df["price"]

# X = X.astype('float')
# y = LabelEncoder().fit_transform(y.astype('str'))

print(X.shape, y.shape)

Series([], dtype: object)
Series([], dtype: object)
(15129, 27) (15129,)


In [107]:
model = LogisticRegression(solver="liblinear")
# cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)
# scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [108]:
def show_results():
    return results.sort_values(by="rmse")

In [109]:
show_results()

,dataset,method,variant,rmse,mse,mae,timestamp
7,original,linear,kbest20_autofeatures,562268.751567,3.161461e+11,375787.001973,2024-06-05T135508
8,original,linear,kbest28_autofeatures,562268.751567,3.161461e+11,375787.001973,2024-06-05T141438
9,original,linear,kbest28_autofeatures,562268.751567,3.161461e+11,375787.001973,2024-06-05T141522
6,original,linear,kbest20,574592.902488,3.301570e+11,381001.812280,2024-06-05T135422
5,original,linear,kbest15,621400.928041,3.861391e+11,409687.407358,2024-06-05T135421
3,original,linear,kbest10,826226.261602,6.826498e+11,524141.924059,2024-06-05T135418
2,original,linear,demo,883418.891027,7.804289e+11,610430.326821,2024-06-05T135408
1,original,linear,demo,883418.891027,7.804289e+11,610430.326821,2024-06-05T133220
4,original,linear,kbest5,966791.661044,9.346861e+11,662190.564664,2024-06-05T135420


In [ ]:

# test_data = predict_and_store('linear_regression_pipeline', model, X)
# mean_absolute_error(train_data['price'], model.predict(X))

In [58]:

# transforms for the feature union
transforms = list()
transforms.append(("mms", MinMaxScaler()))
transforms.append(("ss", StandardScaler()))
transforms.append(("rs", RobustScaler()))
transforms.append(
    ("qt", QuantileTransformer(n_quantiles=100, output_distribution="normal"))
)
transforms.append(
    ("kbd", KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"))
)
transforms.append(("pca", PCA(n_components=7)))
transforms.append(("svd", TruncatedSVD(n_components=7)))
# create the feature union
fu = FeatureUnion(transforms)

In [59]:
model = LogisticRegression(solver="liblinear")
rfe = RFE(estimator=LogisticRegression(solver="liblinear"), n_features_to_select=8)
steps = list()
steps.append(("fu", fu))
steps.append(("rfe", rfe))
steps.append(("m", model))
pipeline = Pipeline(steps=steps)

# cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)
# scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, verbose=1)
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# Autogloun

In [110]:

ld_autogluon = load_split(datetime_to_numeric=False, standard_scaler=False)

X = ld_autogluon.X
y = ld_autogluon.y

Series([], dtype: object)
Series([], dtype: object)


In [111]:
label = "price"
ld_autogluon.train_data[label].describe()

count    1.512900e+04
mean     1.615167e+06
std      1.077826e+06
min      2.250000e+05
25%      9.630000e+05
50%      1.350000e+06
75%      1.923000e+06
max      2.310000e+07
Name: price, dtype: float64

In [112]:
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=X, y=y)

,id,cabins,decks,bathrooms,producer,total_area,crew_area,cabins_area,maintenance_area,crew_area_coef,...,sale_date.day,sale_date.dayofweek,created_date,created_date.year,created_date.month,created_date.dayofweek,refitted_date,refitted_date.year,refitted_date.month,refitted_date.dayofweek
0,1,3,1.0,1,59,364.6,118.0,68.0,50.0,93.0,...,28,5,4310064000000000000,2106,8,6,4310064000000000000,2106,8,6
1,2,3,1.0,3,24,998.3,92.5,54.5,38.0,70.5,...,28,5,4594060800000000000,2115,8,3,4594060800000000000,2115,8,3
2,3,4,2.0,2,24,1793.4,135.5,135.5,0.0,139.0,...,28,5,4573065600000000000,2114,12,5,4573065600000000000,2114,12,5
3,4,3,1.0,2,51,327.5,119.5,72.0,47.5,100.5,...,28,5,4354819200000000000,2108,1,6,4354819200000000000,2108,1,6
4,5,4,1.5,2,41,2144.2,181.5,115.0,66.5,141.5,...,28,5,4507401600000000000,2112,11,1,4507401600000000000,2112,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,15125,3,1.0,2,14,2243.3,70.5,70.5,0.0,147.5,...,10,4,4544208000000000000,2114,1,0,4544208000000000000,2114,1,0
15125,15126,4,2.0,2,20,364.5,101.0,101.0,0.0,104.5,...,10,4,4602009600000000000,2115,11,4,4602009600000000000,2115,11,4
15126,15127,3,1.0,1,43,200.0,95.0,65.0,30.0,94.0,...,10,4,4417977600000000000,2110,1,2,4417977600000000000,2110,1,2
15127,15128,2,1.0,1,50,333.5,69.5,36.0,33.5,46.0,...,20,0,4270579200000000000,2105,5,4,4270579200000000000,2105,5,4


In [113]:
with mlflow.start_run(
    run_name="linear-automlfeatures",
    tags={
        "version": "v1",
        "library": "sklearn",
        "optimization": "AutoMLPipelineFeatureGenerator",
    },
    description="sklearn with AutoMLPipelineFeatureGenerator",
):
    model = LinearRegression().fit(
        auto_ml_pipeline_feature_generator.transform(X=ld_autogluon.X_train),
        ld_autogluon.y_train,
    )
    calculate_metrics(
        ld_autogluon,
        model,
        method="linear",
        variant="automlfeatures",
        y_pred=model.predict(
            auto_ml_pipeline_feature_generator.transform(X=ld_autogluon.X_test)
        ),
    )

2024/06/05 14:18:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2024/06/05 14:18:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a wa

MSE:  493223782573.8531
RMSE:  702298.9267924685
MAE:  481399.6393533197


In [118]:

predictor = None

time_limit = 3600
TIME_LIMIT_BASE=600
presets = "medium_quality"


def execute_autogluon(
    time_limit=3600,
    presets="medium_quality",
    num_bag_folds=5,
    num_bag_sets=3,
    num_stack_levels=3,
    calibrate_decision_threshold=True,
):
    with mlflow.start_run(
        run_name=f"autogluon-{presets}",
        tags={"version": "v1", "library": "autogluon", "optimization": presets},
        description="autogluon",
    ):
        hyperparameters = {
          'GBM': {},
          'XGB': {},
          'CAT': {}
        }

        predictor = TabularPredictor(
            label=label, problem_type="regression", path=f"AutogluonModels/{presets}"
        ).fit(
            ld_autogluon.train_data,
            num_gpus=1,
            presets=presets,
            time_limit=round(time_limit * .9),
            dynamic_stacking=False,
            num_bag_folds=num_bag_folds,
            num_bag_sets=num_bag_sets,
            num_stack_levels=num_stack_levels,
            calibrate_decision_threshold=calibrate_decision_threshold,
            hyperparameters=hyperparameters,
            feature_generator=auto_ml_pipeline_feature_generator,
        )

        extra_hyperparameters = {
          'NN_TORCH': {},
          'FASTAI': {}
        }
        predictor.fit_extra(
            time_limit=round(time_limit * .1),
            hyperparameters=extra_hyperparameters
        )

        calculate_metrics(
            ld_autogluon,
            model,
            method="autogluon",
            variant=f"{presets}_{time_limit}s",
            y_pred=predictor.predict(
                auto_ml_pipeline_feature_generator.transform(X=ld_autogluon.X_test)
            ),
        )
        metrics = predictor.evaluate(ld_autogluon.train_data, silent=True)
        mlflow.log_metrics(metrics)
        predictor.leaderboard(ld_autogluon.train_data)
        predict_and_store(f"autogluon_{presets}", predictor, ld_autogluon.test_data)
        return predictor

In [119]:
medium_quality_predictor = execute_autogluon(presets="medium_quality", time_limit=TIME_LIMIT_BASE)
medium_quality_predictor.evaluate(ld_autogluon.train_data, silent=True)

Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 540s
AutoGluon will save models to "AutogluonModels/medium_quality"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.4
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2
CPU Count:          8
Memory Avail:       42.91 GB / 62.58 GB (68.6%)
Disk Space Avail:   197.85 GB / 848.70 GB (23.3%)
Train Data Rows:    15129
Train Data Columns: 27
Label Column:       price
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
AutoMLPipelineFeatureGenerator is already fit, so the training data will be processed via .transform() instead of .fit_transform().
	Types of features in original data (raw dtype, special dtypes):
		('datetime', []) :  3 | ['sale_date', 'created_date', 'refitted_date']
		('float', [])    : 10 | ['decks'

MSE:  23001677664.308468
RMSE:  151663.0398756021
MAE:  108636.1412343027


2024/06/05 14:41:17 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of fastai. If you encounter errors during autologging, try upgrading / downgrading fastai to a supported version, or try upgrading MLflow.
2024/06/05 14:41:17 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for fastai: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_ABI_DT_RELR' not found (required by /lib/x86_64-linux-gnu/libutil.so.1)


ImportError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_ABI_DT_RELR' not found (required by /lib/x86_64-linux-gnu/libutil.so.1)

In [ ]:
medium_quality_predictor.feature_importance(ld_autogluon.train_data, time_limit=int(0.5 * TIME_LIMIT_BASE))

In [ ]:
medium_quality_predictor.leaderboard(ld_autogluon.train_data)

In [ ]:
high_quality_predictor = execute_autogluon(presets="high_quality", time_limit=TIME_LIMIT_BASE * 2)
high_quality_predictor.leaderboard(ld_autogluon.train_data)

In [72]:
best_quality_predictor = execute_autogluon(presets="best_quality", time_limit=TIME_LIMIT_BASE * 3)
best_quality_predictor.leaderboard(ld_autogluon.train_data)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Beginning AutoGluon training ... Time limit = 5400s
AutoGluon will save models to "AutogluonModels/best_quality"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Apr 28 14:29:16 UTC 2024
CPU Count:          2
Memory Avail:       9.57 GB / 12.67 GB (75.5%)
Disk Space Avail:   43.42 GB / 78.19 GB (55.5%)
Train Data Rows:    7578
Train Data Columns: 27
Label Column:       price
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
AutoMLPipelineFeatureGenerator is already fit, so the training data will be processed via .transform() instead of .fit_transform().
	Useless Original Features (Count: 1): ['sale_age']
		These features carry no predictive signal and 

MSE:  16201293707.141886
RMSE:  127284.30267374641
MAE:  63691.22701599604
Series([], dtype: object)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_r9_BAG_L1,-9.925773e+04,-3.974387e+05,root_mean_squared_error,2.977672,0.802344,457.739947,2.977672,0.802344,457.739947,1,True,16
1,CatBoost_BAG_L2,-1.044102e+05,-2.151500e+05,root_mean_squared_error,76.276327,20.318696,2511.801823,0.625645,0.143081,770.632830,2,True,24
2,CatBoost_r177_BAG_L2,-1.065863e+05,-2.109998e+05,root_mean_squared_error,75.911985,20.254483,1873.859846,0.261303,0.078868,132.690853,2,True,30
3,CatBoost_BAG_L3,-1.139760e+05,-1.902900e+05,root_mean_squared_error,119.654342,36.276709,3796.403368,0.327107,0.138441,488.404920,3,True,37
4,CatBoost_r177_BAG_L3,-1.184230e+05,-1.914816e+05,root_mean_squared_error,119.541133,36.659405,3575.599932,0.213898,0.521137,267.601484,3,True,43
5,LightGBMXT_BAG_L2,-1.219689e+05,-2.685285e+05,root_mean_squared_error,101.571553,29.582593,1846.224735,25.920872,9.406978,105.055742,2,True,21
6,CatBoost_BAG_L1,-1.251806e+05,-3.586282e+05,root_mean_squared_error,0.147097,0.047288,116.917436,0.147097,0.047288,116.917436,1,True,6
7,CatBoost_BAG_L4,-1.299608e+05,-2.075974e+05,root_mean_squared_error,129.182655,41.606514,4829.960499,0.202220,0.081115,236.033252,4,True,50
8,CatBoost_r177_BAG_L1,-1.339661e+05,-3.587582e+05,root_mean_squared_error,0.311015,0.047431,65.445792,0.311015,0.047431,65.445792,1,True,12
9,XGBoost_BAG_L3,-1.374414e+05,-1.825016e+05,root_mean_squared_error,120.480057,36.504326,3346.893401,1.152822,0.366057,38.894953,3,True,40


In [ ]:
# test_data = load_test(datetime_to_numeric=False)
# df = auto_ml_pipeline_feature_generator.transform(test_data)
# test_x = df.drop([], axis=1)
# predict_and_store(f"autogluon_{presets}", medium_quality_predictor, test_x)

In [120]:
show_results()

,dataset,method,variant,rmse,mse,mae,timestamp
11,original,autogluon,medium_quality_600s,151663.039876,2.300168e+10,108636.141234,2024-06-05T144103
7,original,linear,kbest20_autofeatures,562268.751567,3.161461e+11,375787.001973,2024-06-05T135508
9,original,linear,kbest28_autofeatures,562268.751567,3.161461e+11,375787.001973,2024-06-05T141522
8,original,linear,kbest28_autofeatures,562268.751567,3.161461e+11,375787.001973,2024-06-05T141438
6,original,linear,kbest20,574592.902488,3.301570e+11,381001.812280,2024-06-05T135422
5,original,linear,kbest15,621400.928041,3.861391e+11,409687.407358,2024-06-05T135421
10,original,linear,automlfeatures,702298.926792,4.932238e+11,481399.639353,2024-06-05T141855
3,original,linear,kbest10,826226.261602,6.826498e+11,524141.924059,2024-06-05T135418
1,original,linear,demo,883418.891027,7.804289e+11,610430.326821,2024-06-05T133220
2,original,linear,demo,883418.891027,7.804289e+11,610430.326821,2024-06-05T135408


# Optuna + Autogluon

In [121]:
N_TRIALS=5

def execute_autogluon_optuna(
    presets="medium_quality", time_limit=3600, n_trials=10, study_alias=""
):
    with mlflow.start_run(
        run_name="autogluon_optuna",
        tags={"version": "v1", "library": "autogluon", "optimization": "optuna"},
        description="autogluon_optuna",
    ) as parent_run:
        mlflow.log_param("parent", "yes")

        def objective(trial):
            number = trial.number
            with mlflow.start_run(
                run_name=f"autogluon_optuna_trial_{number}",
                description="autogluon_optuna_trial",
                tags={
                    "version": "v1",
                    "library": "autogluon",
                    "optimization": "optuna",
                    "trial": "true",
                },
                nested=True,
            ):
                params = {
                    "num_bag_folds": trial.suggest_int("num_bag_folds", 2, 5),
                    "num_bag_sets": trial.suggest_int("num_bag_sets", 1, 5),
                    "num_stack_levels": trial.suggest_int("num_stack_levels", 1, 5),
                    "calibrate_decision_threshold": trial.suggest_categorical(
                        "calibrate_decision_threshold", [False]
                    ),
                }
                ld = load_split()

                hyperparameters = {
                  'GBM': {},
                  'XGB': {},
                  'CAT': {},
                }
                predictor = TabularPredictor(
                    label=label,
                    problem_type="regression",
                    path=f"AutogluonModels/{study_alias}_{presets}_trial_{number}_n{n_trials}_{time_limit}s",
                ).fit(
                    ld_autogluon.train_data,
                    num_gpus=1,
                    presets=presets,
                    time_limit=time_limit,
                    dynamic_stacking=False,
                    hyperparameters=hyperparameters,
                    feature_generator=auto_ml_pipeline_feature_generator,
                    **params,
                )
                model = predictor
                y_pred = model.predict(ld.X_test)

                calculate_metrics(ld, model, method="autogluon_optuna_trial", variant="trial")

                return mean_squared_error(ld.y_test, y_pred)

        study = optuna.create_study(
            direction="minimize",
            storage="sqlite:///optuna.sqlite3",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(),
            study_name=f"autogluon-regression_{study_alias}_n{n_trials}_{time_limit}s_{presets}",
            load_if_exists=True,
        )
        study.optimize(objective, n_trials=n_trials)

        ld = load_split()
        best_trial = study.best_trial
        # model = TabularPredictor(label=label, problem_type="regression", path=f'AutogluonModels/optuna-{presets}').fit(
        #     ld_autogluon.train_data,
        #     num_gpus=1,
        #     presets=presets,
        #     time_limit=time_limit,
        #     dynamic_stacking=False,
        #     feature_generator=auto_ml_pipeline_feature_generator,
        #     **study.best_params,
        # )
        number = best_trial.number
        model = TabularPredictor.load(f"AutogluonModels/{study_alias}_{presets}_trial_{number}_n{n_trials}_{time_limit}s")
        # model.fit(ld.X, ld.y)
        # y_pred = model.predict(ld.X_test)

        calculate_metrics(ld, model, method="autogluon-optuna", variant=f"{presets}_n{n_trials}_{time_limit}s_{study_alias}")
        predict_and_store(f"autogluon-optuna_{presets}_{number}_n{n_trials}_{time_limit}s", model, ld_autogluon.test_data)

        return model, study

In [122]:
_, study = execute_autogluon_optuna(
    presets="medium_quality", n_trials=N_TRIALS, time_limit=round(TIME_LIMIT_BASE / 6), study_alias="first"
)

[I 2024-06-05 14:43:09,922] A new study created in RDB with name: autogluon-regression_first_n5_100s_medium_quality
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 100s
AutoGluon will save models to "AutogluonModels/first_medium_quality_trial_0_n5_100s"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.4
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2
CPU Count:          8
Memory Avail:       42.03 GB / 62.58 GB (67.2%)
Disk Space Avail:   197.82 GB / 848.70 GB (23.3%)
Train Data Rows:    15129
Train Data Columns: 27
Label Column:       price
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
AutoMLPipelineFeatureGenerator is already fit, so the training data will be processed via .transform() instead of .fit_transform().
	Types of features in original dat

Series([], dtype: object)
Series([], dtype: object)


	-361699.6186	 = Validation score   (-root_mean_squared_error)
	4.81s	 = Training   runtime
	0.5s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 20.26s of the 93.55s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 3 child models (S1F1 - S1F3) | Fitting with ParallelLocalFoldFittingStrategy (3 workers, per: cpus=2, gpus=0, memory=0.09%)
	-344946.3406	 = Validation score   (-root_mean_squared_error)
	16.77s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 2.19s of the 75.49s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 3 child models (S1F1 - S1F3) | Fitting with ParallelLocalFoldFittingStrategy (3 workers, per: cpus=2, gpus=0, memory=0.11%)
	-377864.502	 = Validation score   (-root_mean_squared_error)
	2.41s	 = Training   runtime
	0.18s	 = Validation runtime
Completed 1/5 k-fold baggi

MSE:  2483148791563.0425
RMSE:  1575800.9999879561
MAE:  1228830.5879796967
Series([], dtype: object)


Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 100s
AutoGluon will save models to "AutogluonModels/first_medium_quality_trial_1_n5_100s"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.4
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2
CPU Count:          8
Memory Avail:       42.70 GB / 62.58 GB (68.2%)
Disk Space Avail:   197.76 GB / 848.70 GB (23.3%)
Train Data Rows:    15129
Train Data Columns: 27
Label Column:       price
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
AutoMLPipelineFeatureGenerator is already fit, so the training data will be processed via .transform() instead of .fit_transform().
	Types of features in original data (raw dtype, special dtypes):
		('datetime', []) :  3 | ['sale_date', 'created_date', 'refitted_date']
		('float', 

...

MSE:  2440710802905.3774
RMSE:  1562277.4410793295
MAE:  1220438.4474618412


Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 100s
AutoGluon will save models to "AutogluonModels/first_medium_quality_trial_4_n5_100s"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.4
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2
CPU Count:          8
Memory Avail:       42.94 GB / 62.58 GB (68.6%)
Disk Space Avail:   197.66 GB / 848.70 GB (23.3%)
Train Data Rows:    15129
Train Data Columns: 27
Label Column:       price
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
AutoMLPipelineFeatureGenerator is already fit, so the training data will be processed via .transform() instead of .fit_transform().
	Types of features in original data (raw dtype, special dtypes):
		('datetime', []) :  3 | ['sale_date', 'created_date', 'refitted_date']
		('float', 

Series([], dtype: object)
Series([], dtype: object)


	-357159.1884	 = Validation score   (-root_mean_squared_error)
	9.94s	 = Training   runtime
	1.35s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 53.71s of the 87.04s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (5 workers, per: cpus=1, gpus=0, memory=0.08%)
	-340771.4309	 = Validation score   (-root_mean_squared_error)
	42.99s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 9.38s of the 42.71s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (5 workers, per: cpus=1, gpus=0, memory=0.10%)
	-362355.601	 = Validation score   (-root_mean_squared_error)
	8.35s	 = Training   runtime
	0.46s	 = Validation runtime
Completed 1/2 k-fold bagg

MSE:  2232196076521.2314
RMSE:  1494053.572172441
MAE:  1107363.0659776726
Series([], dtype: object)
Series([], dtype: object)
MSE:  2071940283965.972
RMSE:  1439423.594348089
MAE:  1039568.8409048661
Series([], dtype: object)


In [ ]:
# _, hq_study = execute_autogluon_optuna(
#     presets="high_quality", n_trials=N_TRIALS * 2, time_limit=TIME_LIMIT_BASE * 2, study_alias="first"
# )

In [ ]:
# _, best_study = execute_autogluon_optuna(
#     presets="best_quality", n_trials=N_TRIALS * 3, time_limit=TIME_LIMIT_BASE * 3, study_alias="first"
# )

In [123]:
show_results()[0:20]

,dataset,method,variant,rmse,mse,mae,timestamp
11,original,autogluon,medium_quality_600s,1.516630e+05,2.300168e+10,1.086361e+05,2024-06-05T144103
7,original,linear,kbest20_autofeatures,5.622688e+05,3.161461e+11,3.757870e+05,2024-06-05T135508
9,original,linear,kbest28_autofeatures,5.622688e+05,3.161461e+11,3.757870e+05,2024-06-05T141522
8,original,linear,kbest28_autofeatures,5.622688e+05,3.161461e+11,3.757870e+05,2024-06-05T141438
6,original,linear,kbest20,5.745929e+05,3.301570e+11,3.810018e+05,2024-06-05T135422
5,original,linear,kbest15,6.214009e+05,3.861391e+11,4.096874e+05,2024-06-05T135421
10,original,linear,automlfeatures,7.022989e+05,4.932238e+11,4.813996e+05,2024-06-05T141855
3,original,linear,kbest10,8.262263e+05,6.826498e+11,5.241419e+05,2024-06-05T135418
1,original,linear,demo,8.834189e+05,7.804289e+11,6.104303e+05,2024-06-05T133220
2,original,linear,demo,8.834189e+05,7.804289e+11,6.104303e+05,2024-06-05T135408


In [ ]:
ov.plot_optimization_history(study)

In [ ]:
ov.plot_intermediate_values(study)

In [ ]:
ov.plot_timeline(study)

In [ ]:
ov.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

# XGBoost

* https://github.com/dmlc/xgboost/blob/master/demo/guide-python/sklearn_examples.py

In [ ]:
!pip install xgboost

# bst = xgboost.train(...)
# config = bst.save_config()
# print(config)

In [ ]:


def execute_xgboost(features=None, variant="default"):
    import numpy as np
    import xgboost as xgb
    from numpy import mean, std

    with mlflow.start_run(
        run_name=f"xgboost-{variant}",
        tags={"version": "v1", "library": "xgboost", "optimization": "n/a"},
        description="xgboost",
    ):
        ld = load_split(features=features)
        train_data = ld.X_train

        model = xgb.XGBRegressor(n_jobs=1, tree_method="hist", device="cuda")
        model = model.fit(ld.X_train, ld.y_train)

        calculate_metrics(ld, model, method="xgboost", variant=variant)

        return model, ld.X_train

In [ ]:
_ = execute_xgboost(features=select_features(k=10)[0], variant="kbest10")
_ = execute_xgboost(features=select_features(k=20)[0], variant="kbest20")
_ = execute_xgboost()

show_results()

INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)
Keeping features ['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [02:33:52] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
2024/06/05 02:33:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:36: UserWarning: Setuptools is replacing distutils."


...

INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)
Keeping features ['id', 'sale_date', 'decks', 'producer', 'created_date', 'refitted_date', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'overall_quality', 'created_age', 'refitted_age', 'sale_age']
MSE:  189722324882.6673
RMSE:  435571.26269149955
MAE:  227406.26280568406


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)
MSE:  142231004005.68985
RMSE:  377135.259563051
MAE:  214638.47461582947


,dataset,method,variant,rmse,mse,mae,timestamp
8,original,xgboost,default,377135.259563,1.422310e+11,214638.474616,2024-06-05T023407
7,original,xgboost,kbest20,435571.262691,1.897223e+11,227406.262806,2024-06-05T023402
5,original,linear,kbest20,574592.902487,3.301570e+11,381001.812280,2024-06-05T023327
4,original,linear,kbest15,621400.928041,3.861391e+11,409687.407358,2024-06-05T023327
6,original,xgboost,kbest10,715216.898464,5.115352e+11,401716.023133,2024-06-05T023358
2,original,linear,kbest10,826226.261602,6.826498e+11,524141.924059,2024-06-05T023324
1,original,linear,demo,883418.891027,7.804289e+11,610430.326821,2024-06-05T023320
3,original,linear,kbest5,966791.661044,9.346861e+11,662190.564664,2024-06-05T023326


In [ ]:


def execute_xgboost_cv(features=None, variant="default"):
    import numpy as np
    import xgboost as xgb
    from numpy import mean, std
    from sklearn.model_selection import RepeatedKFold, cross_val_score

    with mlflow.start_run(
        run_name=f"xgboost-cv-{variant}",
        tags={"version": "v1", "library": "xgboost", "optimization": "cv"},
        description="xgboost cross validation",
    ):
        ld = load_split(features=features)
        train_data = ld.X_train

        model = xgb.XGBRegressor(n_jobs=1, tree_method="hist", device="cuda")
        model = model.fit(ld.X, ld.y)

        cv = RepeatedKFold(n_splits=3, n_repeats=10, random_state=1)
        scores = cross_val_score(model, ld.X, ld.y, cv=cv, n_jobs=-1, verbose=1)

        calculate_metrics(ld, model, method="xgboost-cv", variant=variant)
        mlflow.log_metrics({"cv-mean": mean(scores), "cv-std": std(scores)})

        return model, ld.X


# xgb_model = xgb.XGBRegressor(n_jobs=1, tree_method="hist", device="cuda")
# xgb_model.fit(X, y)
# cv = RepeatedKFold(n_splits=3, n_repeats=10, random_state=1)
# scores = cross_val_score(xgb_model, X, y, cv=cv, n_jobs=-1, verbose=1)
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
_ = execute_xgboost_cv(features=select_features(k=10)[0], variant="kbest10")
_ = execute_xgboost_cv(features=select_features(k=20)[0], variant="kbest20")
_ = execute_xgboost_cv()

INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)
Keeping features ['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']


2024/06/05 02:34:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [02:34:32] 

...

MSE:  2763699163213.1064
RMSE:  1662437.7170929161
MAE:  1344822.4949396893


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)


2024/06/05 02:34:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   11.3s finished


MSE:  2623506524927.389
RMSE:  1619724.2126137984
MAE:  1279643.5861492069


In [ ]:
show_results()

,dataset,method,variant,rmse,mse,mae,timestamp
8,original,xgboost,default,3.771353e+05,1.422310e+11,2.146385e+05,2024-06-05T023407
7,original,xgboost,kbest20,4.355713e+05,1.897223e+11,2.274063e+05,2024-06-05T023402
5,original,linear,kbest20,5.745929e+05,3.301570e+11,3.810018e+05,2024-06-05T023327
4,original,linear,kbest15,6.214009e+05,3.861391e+11,4.096874e+05,2024-06-05T023327
6,original,xgboost,kbest10,7.152169e+05,5.115352e+11,4.017160e+05,2024-06-05T023358
2,original,linear,kbest10,8.262263e+05,6.826498e+11,5.241419e+05,2024-06-05T023324
1,original,linear,demo,8.834189e+05,7.804289e+11,6.104303e+05,2024-06-05T023320
3,original,linear,kbest5,9.667917e+05,9.346861e+11,6.621906e+05,2024-06-05T023326
9,original,xgboost-cv,kbest10,1.048696e+06,1.099764e+12,6.353146e+05,2024-06-05T023441
11,original,xgboost-cv,default,1.619724e+06,2.623507e+12,1.279644e+06,2024-06-05T023512


# Optuna + XGBoost

In [ ]:
N_TRIALS=30
xgboost_study = None
xgboost_optuna_model = None
def execute_xgboost_optuna(n_trials):
    with mlflow.start_run(
        run_name="xgboost_optuna",
        tags={"version": "v1", "library": "xgboost", "optimization": "optuna"},
        description="xgboost_optuna",
    ) as parent_run:
        mlflow.log_param("parent", "yes")

        def objective(trial):
            number = trial.number
            with mlflow.start_run(
                run_name=f"xgboost_optuna_trial_{number}",
                description="xgboost_optuna_trial",
                tags={
                    "version": "v1",
                    "library": "xgboost",
                    "optimization": "optuna",
                    "trial": "true",
                },
                nested=True,
            ):
                param = {
                    "max_depth": trial.suggest_int("max_depth", 1, 10),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
                    "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
                    "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                    "gamma": trial.suggest_float("gamma", 0.01, 1.0),
                    "subsample": trial.suggest_float("subsample", 0.01, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 1.0),
                    "reg_alpha": trial.suggest_float("reg_alpha", 0.01, 1.0),
                    "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 1.0),
                    "random_state": trial.suggest_int("random_state", 1, 1000),
                    "device": "cuda",
                    "tree_method": "gpu_hist",
                }
                ld = load_split()

                model = xgb.XGBRegressor(**param)
                model = model.fit(ld.X_train, ld.y_train)
                y_pred = model.predict(ld.X_test)

                calculate_metrics(ld, model, method="xgboost_optuna_trial")

                return mean_squared_error(ld.y_test, y_pred)

        study = optuna.create_study(
            direction="minimize",
            storage="sqlite:///optuna.sqlite3",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(),
            study_name="xgboost-regression",
            load_if_exists=True,
        )
        study.optimize(objective, n_trials=n_trials)

        ld = load_split()
        model = xgboost.XGBRegressor(**study.best_params)
        model.fit(ld.X, ld.y)
        y_pred = model.predict(ld.X_test)
        xgboost_study = study
        xgboost_optuna_model = model

        calculate_metrics(ld, model, method="xgboost-optuna", variant=f"n{n_trials}")
        predict_and_store("xgboost-optuna", xgboost_optuna_model, ld.test_data)
        return model, study

xgboost_optuna_model, xgboost_study = execute_xgboost_optuna(n_trials=N_TRIALS * 5)

[I 2024-06-05 02:35:42,705] A new study created in RDB with name: xgboost-regression
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)


[I 2024-06-05 02:35:48,170] Trial 0 finished with value: 521210841058.68555 and parameters: {'max_depth': 4, 'learning_rate': 0.951207163345817, 'n_estimators': 746, 'min_child_weight': 6, 'gamma': 0.16445845403801215, 'subsample': 0.16443457513284063, 'colsample_bytree': 0.06750277604651747, 'reg_alpha': 0.8675143843171859, 'reg_lambda': 0.6051038616257767, 'random_state': 709}. Best is trial 0 with value: 521210841058.68555.


MSE:  521210841058.68555
RMSE:  721949.3341354957
MAE:  535222.7599760409


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


...

MSE:  103515665742.59532
RMSE:  321738.50522216846
MAE:  197169.4844059815


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)


[2024-06-05 02:55:14 +0000] [634] [INFO] Handling signal: int
[2024-06-05 02:55:14 +0000] [635] [INFO] Worker exiting (pid: 635)
[2024-06-05 02:55:14 +0000] [636] [INFO] Worker exiting (pid: 636)
[2024-06-05 02:55:14 +0000] [639] [INFO] Worker exiting (pid: 639)
[2024-06-05 02:55:14 +0000] [637] [INFO] Worker exiting (pid: 637)
[2024-06-05 02:55:14 +0000] [634] [INFO] Shutting down: Master
[W 2024-06-05 02:55:16,214] Trial 196 failed with parameters: {'max_depth': 4, 'learning_rate': 0.09292162918510953, 'n_estimators': 820, 'min_child_weight': 3, 'gamma': 0.14871748097761683, 'subsample': 0.8005523210429838, 'colsample_bytree': 0.5649652794368724, 'reg_alpha': 0.05197359963429098, 'reg_lambda': 0.9352069212785851, 'random_state': 220} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/tmp/ipykernel_532/248517722.py", line 43, in objective
    y_pred = model.predict(ld.X_test)
  File "/opt/conda/lib/python3.10/site-packages/mlflow/utils/aut

KeyboardInterrupt: 

In [ ]:
plot_intermediate_values(xgboost_study)

In [ ]:
show_results()

# Optuna + LightGBM
* https://practicaldatascience.co.uk/machine-learning/how-to-tune-a-lightgbmclassifier-model-with-optuna
* https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.htmloptuna

In [ ]:
dataset = load_split(datetime_to_numeric=True)

lightgbm_model = None
def execute_lightgbm(features=None, variant="default"):
    with mlflow.start_run(
        run_name="lightgbm_optuna_baseline",
        tags={"version": "v1", "library": "lightgbm", "optimization": "none"},
        description="lightgbm_optuna_baseline",
    ):
        ld = load_split(features=features)
        model = lgbm.LGBMRegressor()
        model.fit(ld.X_train, ld.y_train)

        y_pred = model.predict(ld.X_test)
        lightgbm_model = model

        calculate_metrics(ld, model, method="lightgbm", variant=variant)
        return model


In [ ]:
_ = execute_lightgbm(features=select_features(k=10)[0], variant="kbest10")
_ = execute_lightgbm(features=select_features(k=20)[0], variant="kbest20")
_ = execute_lightgbm()

show_results()

In [ ]:
lightgbm_study = None
lightgbm_optuna_model = None
def execute_lightgbm_optuna(n_trials):
    with mlflow.start_run(
        run_name="lightgbm_optuna",
        tags={"version": "v1", "library": "lightgbm", "optimization": "optuna"},
        description="lightgbm_optuna",
    ) as parent_run:
        mlflow.log_param("parent", "yes")

        def objective(trial):
            number = trial.number
            with mlflow.start_run(
                run_name=f"lightgbm_optuna_trial_{number}",
                description="lightgbm_optuna_trial",
                tags={
                    "version": "v1",
                    "library": "lightgbm",
                    "optimization": "optuna",
                    "trial": "true",
                },
                nested=True,
            ):
                param = {
                    "objective": "regression",
                    "learning_rate": trial.suggest_float(
                        "learning_rate", 1e-2, 0.25, log=True
                    ),
                    "max_depth": trial.suggest_int("max_depth", 1, 9),
                    "n_estimators": trial.suggest_categorical(
                        "n_estimators", [7000, 15000, 20000]
                    ),
                    "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
                    "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 256),
                    "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
                    "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
                    "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
                    "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
                    "metric": "rmse",
                    "verbose": -1,
                    "device": "gpu",
                    "gpu_platform_id": 0,
                    "gpu_device_id": 0,
                }

                ld = load_split()
                model = lgbm.LGBMRegressor(**param)
                model = model.fit(ld.X_train, ld.y_train)
                y_pred = model.predict(ld.X_test)

                calculate_metrics(ld, model, method="xgboost_optuna_trial")

                return mean_squared_error(ld.y_test, y_pred)

        study = optuna.create_study(
            direction="minimize",
            storage="sqlite:///optuna.sqlite3",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(),
            study_name="lightgbm-regression",
            load_if_exists=True,
        )
        study.optimize(objective, n_trials=n_trials)

        ld = load_split()
        model = lgbm.LGBMRegressor(**study.best_params)
        model.fit(ld.X, ld.y)
        y_pred = model.predict(ld.X_test)
        lightgbm_study = study
        lightgbm_optuna_model = model

        calculate_metrics(ld, model, method="lightgbm-optuna", variant=f"n{n_trials}")
        predict_and_store("lightgbm-optuna", lightgbm_optuna_model, ld.test_data)
        return model, study

In [ ]:
lightgbm_optuna_model, lightgbm_study = execute_lightgbm_optuna(n_trials=N_TRIALS * 5)

[I 2024-06-05 03:03:13,996] A new study created in RDB with name: lightgbm-regression
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
2024/06/05 03:03:44 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpisy5tes0/model, flavor: lightgbm). Fall back to return ['lightgbm==4.3.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
[I 2024-06-05 0

MSE:  140176255640.02457
RMSE:  374401.1960985496
MAE:  214443.39078315228


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


...

MSE:  127625754814.43506
RMSE:  357247.46999025065
MAE:  218892.2721802748


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Series([], dtype: object)
Series([], dtype: object)


[2024-06-05 03:05:21 +0000] [3929] [INFO] Handling signal: int
[2024-06-05 03:05:21 +0000] [3932] [INFO] Worker exiting (pid: 3932)
[2024-06-05 03:05:21 +0000] [3930] [INFO] Worker exiting (pid: 3930)
[2024-06-05 03:05:21 +0000] [3931] [INFO] Worker exiting (pid: 3931)
[2024-06-05 03:05:21 +0000] [3933] [INFO] Worker exiting (pid: 3933)
[2024-06-05 03:05:21 +0000] [3929] [INFO] Shutting down: Master
[W 2024-06-05 03:05:40,946] Trial 2 failed with parameters: {'learning_rate': 0.04340687070008209, 'max_depth': 8, 'n_estimators': 20000, 'lambda_l1': 2.6185068507773707e-08, 'lambda_l2': 0.0029369981104377003, 'num_leaves': 45, 'feature_fraction': 0.43903095579116774, 'bagging_fraction': 0.9693313223519999, 'bagging_freq': 7, 'min_child_samples': 82} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/tmp/ipykernel_532/2972776057.py", line 49, in objective
    model = model.fit(ld.X_train, ld.y_train)
  File "/opt/conda/lib/python3.10/site-packa

KeyboardInterrupt: 

In [ ]:
show_results()

# cuML

* https://docs.rapids.ai/api/cuml/stable/cuml_intro/#where-possible-match-the-scikit-learn-api

# AutoXGB

* https://github.com/abhishekkrthakur/autoxgb

In [ ]:
# !pip install autoxgb

In [ ]:

# from autoxgb import AutoXGB


# # required parameters:
# train_filename = "train.csv"
# output = "autoxgb-output"

# # optional parameters
# test_filename = "test.csv"
# task = None
# idx = None
# targets = ["price"]
# features = None
# categorical_features = None
# use_gpu = False
# num_folds = 5
# seed = 42
# num_trials = 100
# time_limit = 360
# fast = True

# # Now its time to train the model!
# axgb = AutoXGB(
#     train_filename=train_filename,
#     output=output,
#     test_filename=test_filename,
#     task=task,
#     idx=idx,
#     targets=targets,
#     features=features,
#     categorical_features=categorical_features,
#     use_gpu=use_gpu,
#     num_folds=num_folds,
#     seed=seed,
#     num_trials=num_trials,
#     time_limit=time_limit,
#     fast=fast,
# )

In [ ]:
# axgb.train()

# Sample

In [ ]:
sample_submission

In [ ]:
! pip install -U pip

# Notebooks with GPU
* https://www.kaggle.com/ (30 hours)
* https://colab.research.google.com/ (free-tier w GPU)
* https://www.paperspace.com/pricing (free-tier no GPU)
* https://saturncloud.io/plans/saturn_cloud_plans/ (free-tier w GPU?, Waitlist)
* https://deepnote.com/ (free-tier no GPU)
* https://studiolab.sagemaker.aws/login (waitlist, free-tier with GPU)


# MLFlow Proxy

In [ ]:


# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2VwE0ghPzEaPvlzIt9tdltGLFRZ_6nyxEMvYDUGjwfsW8A3xh"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5555", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

In [ ]:
# run = wandb.init(project=WANDB_PROJECT)


# ds_art = wandb.Artifact(name="original", type="dataset", description="Original dataset")

# # Attach our processed data to the Artifact
# ds_art.add_file(train_path)
# ds_art.add_file(test_path)

# table = wandb.Table(dataframe=load_train().sample(1000))
# wandb.log({"dataset": table})


# run.log_artifact(ds_art)

# run.finish()

In [1]:

!apt-get install -y libboost-all-dev
!pip uninstall -y lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Found existing installation: lightgbm 4.1.0
Uninstalling lightgbm-4.1.0:
  Successfully uninstalled lightgbm-4.1.0
Cloning into 'LightGBM'...
remote: Enumerating objects: 32524, done.
remote: Counting objects: 100% (7869/7869), done.
remote: Compressing objects: 100% (1190/1190), done.
remote: Total 32524 (delta 7299), reused 6893 (delta 6652), pack-reused 24655
Receiving objects: 100% (32524/32524), 21.90 MiB | 17.50 MiB/s, done.
Resolving deltas: 100% (24326/24326), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_d

In [2]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_3_0
-- Looking for CL_VERSION_3_0 - found
-- Found OpenCL: /usr/local/cuda/lib64/libOpenCL.so (found version "3.0") 
-- OpenCL include directory: /usr/local/cuda/include
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.74.0/BoostConfig.cmake (found suitable version "1.74.0", minimum required is "1.56.0") found 

rm: cannot remove 'build': No such file or directory
In file included from /content/LightGBM/external_libs/compute/include/boost/compute/cl.hpp:19,
                 from /content/LightGBM/external_libs/compute/include/boost/compute/config.hpp:16,
                 from /content/LightGBM/external_libs/compute/include/boost/compute/buffer.hpp:14,
                 from /content/LightGBM/external_libs/compute/include/boost/compute/core.hpp:18,
                 from /content/LightGBM/src/treelearner/gpu_tree_learner.h:33,
                 from /content/LightGBM/src/treelearner/parallel_tree_learner.h:15,
                 from /content/LightGBM/src/treelearner/data_parallel_tree_learner.cpp:9:
/usr/local/cuda/include/CL/cl.h:26:104: note: ‘#pragma message: cl_version.h: CL_TARGET_OPENCL_VERSION is not defined. Defaulting to 300 (OpenCL 3.0)’
   26 | #pragma message("cl_version.h: CL_TARGET_OPENCL_VERSION is not defined. Defaulting to 300 (OpenCL 3.0)")
      |                                 

In [1]:
%%bash
cd LightGBM/python-package/;
pip uninstall -y lightgbm
pip install lightgbm \
  --config-settings=cmake.define.USE_GPU=ON \
  --config-settings=cmake.define.OpenCL_INCLUDE_DIR="/usr/local/cuda/include/" \
  --config-settings=cmake.define.OpenCL_LIBRARY="/usr/local/cuda/lib64/libOpenCL.so"

Found existing installation: lightgbm 4.3.0
Uninstalling lightgbm-4.3.0:
  Successfully uninstalled lightgbm-4.3.0
  Using cached lightgbm-4.3.0-py3-none-manylinux_2_28_x86_64.whl (3.1 MB)


In [2]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd


In [ ]:
!rm -r LightGBM